In [ ]:
import pandas as pd
from sodapy import Socrata
import holidays

In [ ]:
results_df = pd.read_csv("./BaseLiquor.csv")

results_df.columns
results_df.dtypes

results_df['sale_dollars'] = results_df['sale_dollars'].astype(float)
results_df['sale_bottles'] = results_df['sale_bottles'].astype(float)
results_df.dtypes

invoice_line_no                 object
date                            object
store                            int64
name                            object
address                         object
city                            object
zipcode                         object
store_location                  object
county_number                  float64
county                          object
category                       float64
category_name                   object
vendor_no                        int64
vendor_name                     object
itemno                           int64
im_desc                         object
pack                             int64
bottle_volume_ml                 int64
state_bottle_cost              float64
state_bottle_retail            float64
sale_bottles                   float64
sale_dollars                   float64
sale_liters                    float64
sale_gallons                   float64
:@computed_region_3r5t_5243    float64
:@computed_region_wnea_7q

In [ ]:
# Data Aggregation

# Shrinking the categories of Alcohol types
category_mapping = {
    'WHISKEY LIQUEUR': 'Whiskies',
    'STRAIGHT RYE WHISKIES': 'Whiskies',
    'TENNESSEE WHISKIES': 'Whiskies',
    'STRAIGHT BOURBON WHISKIES': 'Whiskies',
    'CANADIAN WHISKIES': 'Whiskies',
    'BLENDED WHISKIES': 'Whiskies',
    'SINGLE MALT SCOTCH': 'Whiskies',
    'SCOTCH WHISKIES': 'Whiskies',
    'IRISH WHISKIES': 'Whiskies',
    'CORN WHISKIES': 'Whiskies',
    'JAPANESE WHISKY': 'Whiskies',
    'SINGLE BARREL BOURBON WHISKIES': 'Whiskies',
    'BOTTLED IN BOND BOURBON': 'Whiskies',

    'VODKA 80 PROOF': 'Vodkas',
    'IMPORTED VODKAS': 'Vodkas',
    'VODKA FLAVORED': 'Vodkas',
    '100 PROOF VODKA': 'Vodkas',
    'IMPORTED VODKA - MISC': 'Vodkas',
    'OTHER PROOF VODKA': 'Vodkas',
    'LOW PROOF VODKA': 'Vodkas',

    'PUERTO RICO & VIRGIN ISLANDS RUM': 'Rums',
    'SPICED RUM': 'Rums',
    'FLAVORED RUM': 'Rums',
    'JAMAICA RUM': 'Rums',
    'BARBADOS RUM': 'Rums',

    'COFFEE LIQUEURS': 'Liqueurs',
    'CREAM LIQUEURS': 'Liqueurs',
    'MISC. IMPORTED CORDIALS & LIQUEURS': 'Liqueurs',
    'MISC. AMERICAN CORDIALS & LIQUEURS': 'Liqueurs',
    'AMARETTO - IMPORTED': 'Liqueurs',
    'IMPORTED AMARETTO': 'Liqueurs',
    'ANISETTE': 'Liqueurs',
    'TRIPLE SEC': 'Liqueurs',

    'PEPPERMINT SCHNAPPS': 'Schnapps',
    'APPLE SCHNAPPS': 'Schnapps',
    'BUTTERSCOTCH SCHNAPPS': 'Schnapps',
    'MISCELLANEOUS SCHNAPPS': 'Schnapps',
    'PEACH SCHNAPPS': 'Schnapps',
    'GRAPE SCHNAPPS': 'Schnapps',
    'RASPBERRY SCHNAPPS': 'Schnapps',
    'ROOT BEER SCHNAPPS': 'Schnapps',
    'CINNAMON SCHNAPPS': 'Schnapps',
    'STRAWBERRY SCHNAPPS': 'Schnapps',
    'TROPICAL FRUIT SCHNAPPS': 'Schnapps',
    'SPEARMINT SCHNAPPS': 'Schnapps',
    'WATERMELON SCHNAPPS': 'Schnapps',
    'SCHNAPPS - IMPORTED': 'Schnapps',

    'BLACKBERRY BRANDIES': 'Brandies',
    'IMPORTED GRAPE BRANDIES': 'Brandies',
    'AMERICAN GRAPE BRANDIES': 'Brandies',
    'APRICOT BRANDIES': 'Brandies',
    'PEACH BRANDIES': 'Brandies',
    'CHERRY BRANDIES': 'Brandies',
    'MISCELLANEOUS  BRANDIES': 'Brandies',

    'IMPORTED DRY GINS': 'Gins',
    'AMERICAN DRY GINS': 'Gins',
    'FLAVORED GINS': 'Gins',

    'DISTILLED SPIRITS SPECIALTY': 'Specialty/Other',
    'AMERICAN COCKTAILS': 'Specialty/Other',
    'AMERICAN SLOE GINS': 'Specialty/Other',
    'AMERICAN AMARETTO': 'Specialty/Other',
    'DECANTERS & SPECIALTY PACKAGES': 'Specialty/Other',
    'GREEN CREME DE MENTHE': 'Specialty/Other',
    'DARK CREME DE CACAO': 'Specialty/Other',
    'WHITE CREME DE CACAO': 'Specialty/Other',
    'CREME DE ALMOND': 'Specialty/Other',
    'WHITE CREME DE MENTHE': 'Specialty/Other',
    'ROCK & RYE': 'Specialty/Other',
    'HIGH PROOF BEER - AMERICAN': 'Specialty/Other',
    'AMERICAN ALCOHOL': 'Specialty/Other',
}


results_df['AlcFamily'] = results_df['category_name'].map(category_mapping)

In [ ]:
# Aggregating Dataset
columns_to_convert = [
    'bottle_volume_ml',
    'state_bottle_cost',
    'state_bottle_retail',
    'sale_bottles',
    'sale_dollars',
    'sale_liters',
    'sale_gallons'
]

results_df[columns_to_convert] = results_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')


# Columns to aggregate
agg_columns = [
    'bottle_volume_ml',
    'state_bottle_cost',
    'state_bottle_retail',
    'sale_bottles',
    'sale_dollars',
    'sale_liters',
    'sale_gallons'
]

results_df['date'] = pd.to_datetime(results_df['date'])

results_df['Year'] = results_df['date'].dt.year
results_df['Month'] = results_df['date'].dt.month

aggregated_df = results_df.groupby(['AlcFamily', 'Month', 'Year', 'county'])[agg_columns].mean().reset_index()

aggregated_df['date'] = pd.to_datetime(aggregated_df[['Year', 'Month']].assign(day=1))

print(aggregated_df)


In [ ]:
#EDA / FE Aggregated DF

aggregated_df['is_holiday_season'] = aggregated_df['Year'].isin([10, 11, 12, 1])
aggregated_df['days_in_month'] = aggregated_df['date'].dt.days_in_month

def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

aggregated_df['season'] = aggregated_df['Month'].apply(get_season)

aggregated_df = aggregated_df.sort_values(by=['AlcFamily', 'county', 'Year', 'Month'])

aggregated_df['prev_year_sale_bottles'] = aggregated_df.groupby(['AlcFamily', 'county'])['sale_bottles'].shift(12)

aggregated_df['3m_rolling_average'] = aggregated_df.groupby(['AlcFamily', 'county'])['sale_bottles'].transform(lambda x: x.shift(1).rolling(window=3, min_periods=1).mean())

aggregated_df= aggregated_df[aggregated_df['Year'] > 2013]

print(aggregated_df)


In [ ]:
import matplotlib.pyplot as plt

# Create some visualizations for different features

# 1. Boxplot to visualize sale_bottles during holiday season vs non-holiday season
plt.figure(figsize=(8, 5))
aggregated_df.boxplot(column='sale_bottles', by='is_holiday_season')
plt.title('Sale Bottles: Holiday Season vs Non-Holiday Season')
plt.suptitle('')
plt.xlabel('Holiday Season')
plt.ylabel('Sale Bottles')
plt.show()

# 2. Line plot of rolling 3-month average sales over time
plt.figure(figsize=(10, 6))
for alc_family in aggregated_df['AlcFamily'].unique():
    alc_data = aggregated_df[aggregated_df['AlcFamily'] == alc_family]
    plt.plot(alc_data['date'], alc_data['3m_rolling_average'], label=alc_family)

plt.title('Rolling 3-Month Average Sales by AlcFamily')
plt.xlabel('Date')
plt.ylabel('Rolling 3M Avg Sales')
plt.legend(loc='upper left')
plt.show()

# 3. Boxplot to visualize sale_bottles by season
plt.figure(figsize=(8, 5))
aggregated_df.boxplot(column='sale_bottles', by='season')
plt.title('Sale Bottles by Season')
plt.suptitle('')
plt.xlabel('Season')
plt.ylabel('Sale Bottles')
plt.show()

# 4. Line plot to visualize effect of days in month on sale_bottles
plt.figure(figsize=(10, 6))
plt.scatter(aggregated_df['days_in_month'], aggregated_df['sale_bottles'], alpha=0.5)
plt.title('Effect of Days in Month on Sale Bottles')
plt.xlabel('Days in Month')
plt.ylabel('Sale Bottles')
plt.show()

# 5. Line plot to visualize the effect of previous year's sales on current year
plt.figure(figsize=(10, 6))
plt.scatter(aggregated_df['prev_year_sale_bottles'], aggregated_df['sale_bottles'], alpha=0.5)
plt.title('Previous Year Sales vs Current Year Sales')
plt.xlabel('Previous Year Sale Bottles')
plt.ylabel('Current Year Sale Bottles')
plt.show()

Additional DF Exploration and merging

In [ ]:
# Personal Income 1997 - 2022
PersonalIncDF = pd.read_csv("./PersonalIncome.csv")

print(PersonalIncDF.columns)

PersonalIncDF['Month'] = PersonalIncDF['Date'].apply(lambda x: x.split("/")[0])
PersonalIncDF['Year'] = PersonalIncDF['Date'].apply(lambda x: x.split("/")[2])

reshaped_df = PersonalIncDF.pivot_table(index=['Year', 'Month', 'Name'],
                                        columns='Variable',
                                        values='Value').reset_index()

reshaped_df.columns.name = None

reshaped_df.rename(columns={
    'Personal income': 'Personal Income',
    'Per capita personal income': 'Per Capita Income',
    'Name': 'County'
}, inplace=True)

aggregated_df.rename(columns={'county': 'County', 'year': 'Year', 'month': 'Month'}, inplace=True)

aggregated_df['Year'] = aggregated_df['Year'].astype(int)
aggregated_df['Month'] = aggregated_df['Month'].astype(int)

reshaped_df['Year'] = reshaped_df['Year'].astype(int)
reshaped_df['Month'] = reshaped_df['Month'].astype(int)

aggregated_df['County'] = aggregated_df['County'].astype(str)
reshaped_df['County'] = reshaped_df['County'].astype(str)

aggregated_df['County'] = aggregated_df['County'].str.strip().str.lower()
reshaped_df['County'] = reshaped_df['County'].str.strip().str.lower()

merged_df = pd.merge(aggregated_df, reshaped_df, how='inner', on=['Year', 'Month', 'County'])

In [ ]:
# Unemployment

UnemploymentDF = pd.read_csv("./Unemployment.csv")

UnemploymentDF = UnemploymentDF[UnemploymentDF['AREA TYPE'] == "County"]

def extract_county(Area_Name):
    return Area_Name.split()[0].lower()

UnemploymentDF['County'] = UnemploymentDF['AREA NAME'].apply(extract_county)

UnemploymentDF['MONTH'] = pd.to_datetime(UnemploymentDF['MONTH'], format='%B').dt.month

UnemploymentDF.rename(columns={'MONTH': 'Month', 'YEAR': 'Year'}, inplace=True)

UnemploymentMerge = UnemploymentDF[['County', 'Month', 'Year', 'LABORFORCE', 'EMPLOYMENT', 'UNEMPLOYMENT', 'UNEMPLOYMENT RATE']]

merged_df = pd.merge(merged_df, UnemploymentMerge, how='inner', on=['Year', 'Month', 'County'])
print(merged_df)

In [ ]:
# Age Demographics
AgeDF = pd.read_csv("./AgeDemographics.csv")

AgeDF = AgeDF[(AgeDF['Type'] == "county") & (AgeDF['Sex'] == "Both")]

def extract_county_age(Area_Name):
    return Area_Name.split()[0].lower()

def extract_year(Date):
    return int(Date.split("/")[2])
AgeDF['County'] = AgeDF['Name'].apply(extract_county_age)

AgeDF['Year'] = AgeDF["Data Collection End Date"].apply(extract_year)

AgeDFMerge = AgeDF[['County', "Year", "Value", "Change Rate"]]

merged_df = pd.merge(merged_df, AgeDFMerge, how='inner', on=['Year', 'County'])

In [ ]:
# Population
PopDF = pd.read_csv("./CountyPopulation.csv")

def extract_county_name_population(County):
    return County.split()[0].lower()

def extract_year_population(Year):
    return int(Year.split()[2])

PopDF['County'] = PopDF["County"].apply(extract_county_name_population)

PopDF['Year'] = PopDF["Year"].apply(extract_year_population)

PopDFMerge = PopDF[["County", "Year", "Population"]]

merged_df = pd.merge(merged_df, PopDFMerge, how='inner', on=['Year', 'County'])